# MVP Demo
----
Take input of three form:
- Laptop camera
- Phone Camera
- Video 
Recognise Pose using:
- Start with YOLO-POSE for animal (fast and simple)
- Once Yolo works switch to DeepLabCut for accuracy

接下来的动作： A. 把 YOLO-Pose 替换成真正“动物关键点模型”（我帮你找，并帮你 fine-tune）
B. 给你完整的动作分类 Notebook（坐/卧/站/走）
C. 训练数据采集 + 标签工具（半自动）
D. 构建你的项目文件结构（notebooks / src / models / features）

In [1]:
print("hello world")

hello world


## Input source

In [ ]:
import cv2
from ultralytics import YOLO

# YOLO-Pose 模型（有动物版本）
model = YOLO("yolov8n-pose.pt")   # 先用人类姿态占位，也可以换动物模型

# 三种输入方式：随便换
USE_CAMERA = True
VIDEO_PATH = "dog_video.mp4"
IP_CAMERA = "http://<your_phone_ip>:8080/video"  # 用 IP Webcam App

if USE_CAMERA:
    cap = cv2.VideoCapture(0)
elif VIDEO_PATH:
    cap = cv2.VideoCapture(VIDEO_PATH)
else:
    cap = cv2.VideoCapture(IP_CAMERA)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 运行 YOLO-Pose
    results = model(frame, verbose=False)[0]

    # 展示带关键点的画面
    annotated = results.plot()
    cv2.imshow("Dog Pose", annotated)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


AttributeError: module 'mediapipe.python.solutions' has no attribute 'dog'

因为 Notebook 不能像窗口那样实时更新，所以我们：
用 clear_output(wait=True) 清除上次画面
用 display() 再输出新的图像
帧率大概 10 FPS，不会特别流畅，但完全能观测骨架点
这是 Notebook 实时视频的正常方式。

A. 获取 Feature Vector 提取代码（关键点 → 角度、重心、身体姿势）
B. 获取 动作分类模型 pipeline（坐/卧/走/站）
C. 或继续优化 Notebook 视频流？

## Building feature vector

### Extract Features

In [ ]:
import numpy as np

def extract_feature_vector(keypoints):
    """
    keypoints: shape (N, 3) → [x, y, confidence]
    return: 1D feature vector
    """

    keypoints = np.array(keypoints)

    # 1. 归一化坐标（防止狗太远或太近）
    xs = keypoints[:, 0]
    ys = keypoints[:, 1]

    # 重心
    cx = np.mean(xs)
    cy = np.mean(ys)

    # 以重心为中心
    xs -= cx
    ys -= cy

    # 2. 关键点距离（描述姿态形状）
    dists = np.sqrt(xs**2 + ys**2)

    # 3. 关键点角度（描述方向和动作）
    angles = np.arctan2(ys, xs)

    # 4. 最终特征向量
    feature_vector = np.concatenate([xs, ys, dists, angles])

    return feature_vector


### Engineer features

Will output `Feature vector: (4*N,)`


In [ ]:
for result in results:
    if result.keypoints is not None:
        kpts = result.keypoints.xy[0].cpu().numpy()  # Nx2
        conf = result.keypoints.conf[0].cpu().numpy()  # N
        keypoints = np.hstack([kpts, conf.reshape(-1, 1)])  # Nx3

        feature_vec = extract_feature_vector(keypoints)
        print("Feature vector:", feature_vec.shape)
